In [1]:
import json
import datetime
import pandas as pd
import collections

In [126]:
def strp(date_string):
    return datetime.datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S.%f') 

def get_type(events, type, attribute_name, occurance=1):
    c_occurance = 0
    f_event = None
    for e in events:
        type_object = e["type"]
        if type == type_object: 
            c_occurance += 1
            f_event = e
            if c_occurance < occurance: continue 
            return e[attribute_name]
    if f_event is not None:
        if attribute_name in f_event:
            return f_event[attribute_name]
    return None

def list_elements_containes(name, names):
    for n in names:
        if name in n: return n
    return None 

def convert_and_save(filename: str, to_dataframe):
    excell_filename = filename.replace(".json", ".xlsx")
    df = to_dataframe(filename)

    assert isinstance(df, pd.DataFrame)

    print("Saving to", excell_filename)
    df.to_excel(excell_filename)


# Native Experiment JSON to Excell file

In [111]:
def native_single_deployment_to_dataframe(filename):
    with open(filename, "r") as f:
        experiments = json.load(f)
    
    assert experiments[0]["single_deployment"] # Only one deployment multiple pods

    rows = []
    for e in experiments:
        data = e["data"]

        framework = "native"
        num_deployments = 1
        num_pods = e["pod_count"]
        num_iterations = e["num_iterations"]
        current_iteration = e["iteration"]
        api_call_datetime = data["time_before_create"]

        # Deployment specific
        deployment_name = get_type(data["deployment_events"], "created", "name")
        deployment_creation_date = get_type(data["deployment_events"], "created", "time")
        deployment_schedule_date = get_type(data["deployment_events"], "modified", "time", 2)
        deployment_deletion_date = get_type(data["deployment_events"], "deleted", "time")

        for pod_name in data["pod_events"]:
            pod_name = get_type(data["pod_events"][pod_name], "created", "name")
            pod_creation_date = get_type(data["pod_events"][pod_name], "created", "time")
            pod_schedule_date = get_type(data["pod_events"][pod_name], "modified", "time")
            pod_deletion_date = get_type(data["pod_events"][pod_name], "deleted", "time")

            row = {
                "framework": framework,
                "num_deployments": num_deployments,
                "num_pods": num_pods,
                "current_iteration": current_iteration,
                "num_iterations": num_iterations,
                "api_call_datetime": api_call_datetime,

                "deployment_name": deployment_name,
                "deployment_creation_date": deployment_creation_date,
                "deployment_schedule_date": deployment_schedule_date,
                "deployment_deletion_date": deployment_deletion_date,

                "pod_name": pod_name,
                "pod_creation_date": pod_creation_date,
                "pod_schedule_date": pod_schedule_date,
                "pod_deletion_date": pod_deletion_date,
            }
            rows.append(row)
    
    return pd.DataFrame(rows)

In [112]:
df = native_single_deployment_to_dataframe("jsons/native1/20230326_113324.json")

In [116]:
def native_multi_deployment_to_dataframe(filename):
    with open(filename, "r") as f:
        experiments = json.load(f)
    
    assert not experiments[0]["single_deployment"] # Only one deployment multiple pods

    rows = []
    for e in experiments:
        data = e["data"]

        framework = "native"
        num_deployments = e["pod_count"]
        num_pods = 1
        num_iterations = e["num_iterations"]
        current_iteration = e["iteration"]
        api_call_datetime = data["time_before_create"]

        # Deployment specific
        for deployment_name in data["deployment_events"]:

            deployment_name = get_type(data["deployment_events"][deployment_name], "created", "name")
            deployment_creation_date = get_type(data["deployment_events"][deployment_name], "created", "time")
            deployment_schedule_date = get_type(data["deployment_events"][deployment_name], "modified", "time", 2)
            deployment_deletion_date = get_type(data["deployment_events"][deployment_name], "deleted", "time")

            pod_name = list_elements_containes(deployment_name, data["pod_events"].keys())

            # When we haven't received any event info from the pod so we simply skip
            if pod_name is None:
                continue

            pod_name = get_type(data["pod_events"][pod_name], "created", "name")
            pod_creation_date = get_type(data["pod_events"][pod_name], "created", "time")
            pod_schedule_date = get_type(data["pod_events"][pod_name], "modified", "time")
            pod_deletion_date = get_type(data["pod_events"][pod_name], "deleted", "time")

            row = {
                "framework": framework,
                "num_deployments": num_deployments,
                "num_pods": num_pods,
                "current_iteration": current_iteration,
                "num_iterations": num_iterations,
                "api_call_datetime": api_call_datetime,

                "deployment_name": deployment_name,
                "deployment_creation_date": deployment_creation_date,
                "deployment_schedule_date": deployment_schedule_date,
                "deployment_deletion_date": deployment_deletion_date,

                "pod_name": pod_name,
                "pod_creation_date": pod_creation_date,
                "pod_schedule_date": pod_schedule_date,
                "pod_deletion_date": pod_deletion_date,
            }
            rows.append(row)
    
    return pd.DataFrame(rows)

In [117]:
df = native_multi_deployment_to_dataframe("jsons/native1/20230326_053008.json")

# EdgeNetFed Experiment JSON to Excell file

In [100]:
def edgenetfed_single_deployment_to_dataframe(filename):
    with open(filename, "r") as f:
        experiments = json.load(f)
    
    assert experiments[0]["single_deployment"] # Only one deployment multiple pods

    rows = []
    for e in experiments:
        data = e["data"]

        framework = "native"
        num_deployments = 1
        num_pods = e["pod_count"]
        num_iterations = e["num_iterations"]
        current_iteration = e["iteration"]
        api_call_datetime = data["time_before_create"]

        deployment_name = get_type(data["deployment_events"], "created", "name")
        deployment_creation_worker2_date = get_type(data["deployment_events"], "created", "time")
        deployment_schedule_worker2_date = get_type(data["deployment_events"], "modified", "time", 2)
        deployment_deletion_worker2_date = get_type(data["deployment_events"], "deleted", "time")

        selectivedeployment_name = get_type(data["selectivedeployment_events_worker1"], "created", "name")
        
        selectivedeployment_creation_worker1_date = get_type(data["selectivedeployment_events_worker1"], "created", "time")
        # selectivedeployment_schedule_worker1_date = get_type(data["selectivedeployment_events_worker1"], "modified", "time", 2)
        selectivedeployment_deletion_worker1_date = get_type(data["selectivedeployment_events_worker1"], "deleted", "time")

        selectivedeployment_creation_worker2_date = get_type(data["selectivedeployment_events_worker2"], "created", "time")
        # selectivedeployment_schedule_worker2_date = get_type(data["selectivedeployment_events_worker2"], "modified", "time", 2)
        selectivedeployment_deletion_worker2_date = get_type(data["selectivedeployment_events_worker2"], "deleted", "time")

        # Deployment specific
        for pod_name in data["pod_events"]:
            pod_creation_date = get_type(data["pod_events"][pod_name], "created", "time")
            pod_schedule_date = get_type(data["pod_events"][pod_name], "modified", "time")
            pod_deletion_date = get_type(data["pod_events"][pod_name], "deleted", "time")

            row = {
                "framework": framework,
                "num_deployments": num_deployments,
                "num_pods": num_pods,
                "current_iteration": current_iteration,
                "num_iterations": num_iterations,
                "api_call_datetime": api_call_datetime,

                "selectivedeployment_name": selectivedeployment_name,
                "deployment_name": deployment_name,
                "pod_name": pod_name,

                "deployment_creation_worker2_date": deployment_creation_worker2_date,
                "deployment_schedule_worker2_date": deployment_schedule_worker2_date,
                "deployment_deletion_worker2_date": deployment_deletion_worker2_date,

                "selectivedeployment_creation_worker1_date": selectivedeployment_creation_worker1_date,
                "selectivedeployment_deletion_worker1_date": selectivedeployment_deletion_worker1_date,

                "selectivedeployment_creation_worker2_date": selectivedeployment_creation_worker2_date,
                "selectivedeployment_deletion_worker2_date": selectivedeployment_deletion_worker2_date,

                "pod_creation_date": pod_creation_date,
                "pod_schedule_date": pod_schedule_date,
                "pod_deletion_date": pod_deletion_date,
            }
            rows.append(row)
    
    return pd.DataFrame(rows)

In [101]:
df = edgenetfed_single_deployment_to_dataframe("jsons/edgenetfed1/20230330_123916.json")

In [124]:
def edgenetfed_multi_deployment_to_dataframe(filename):
    with open(filename, "r") as f:
        experiments = json.load(f)
    
    assert not experiments[0]["single_deployment"] # Only one deployment multiple pods

    rows = []
    for e in experiments:
        data = e["data"]

        framework = "native"
        num_deployments = e["pod_count"]
        num_pods = 1
        num_iterations = e["num_iterations"]
        current_iteration = e["iteration"]
        api_call_datetime = data["time_before_create"]

        for selectivedeployment_name in data["selectivedeployment_events_worker1"].keys():
            deployment_name = list_elements_containes(selectivedeployment_name, data["deployment_events"].keys())
            pod_name = list_elements_containes(deployment_name, data["pod_events"].keys())

            if pod_name is None or deployment_name is None:
                continue

            deployment_creation_worker2_date = get_type(data["deployment_events"][deployment_name], "created", "time")
            deployment_schedule_worker2_date = get_type(data["deployment_events"][deployment_name], "modified", "time", 2)
            deployment_deletion_worker2_date = get_type(data["deployment_events"][deployment_name], "deleted", "time")
            
            selectivedeployment_creation_worker1_date = get_type(data["selectivedeployment_events_worker1"][selectivedeployment_name], "created", "time")
            # selectivedeployment_schedule_worker1_date = get_type(data["selectivedeployment_events_worker1"][selectivedeployment_name], "modified", "time", 2)
            selectivedeployment_deletion_worker1_date = get_type(data["selectivedeployment_events_worker1"][selectivedeployment_name], "deleted", "time")

            selectivedeployment_creation_worker2_date = get_type(data["selectivedeployment_events_worker2"][selectivedeployment_name], "created", "time")
            # selectivedeployment_schedule_worker2_date = get_type(data["selectivedeployment_events_worker2"][selectivedeployment_name], "modified", "time", 2)
            selectivedeployment_deletion_worker2_date = get_type(data["selectivedeployment_events_worker2"][selectivedeployment_name], "deleted", "time")

            # Deployment specific
            pod_creation_date = get_type(data["pod_events"][pod_name], "created", "time")
            pod_schedule_date = get_type(data["pod_events"][pod_name], "modified", "time")
            pod_deletion_date = get_type(data["pod_events"][pod_name], "deleted", "time")

            row = {
                "framework": framework,
                "num_deployments": num_deployments,
                "num_pods": num_pods,
                "current_iteration": current_iteration,
                "num_iterations": num_iterations,
                "api_call_datetime": api_call_datetime,

                "selectivedeployment_name": selectivedeployment_name,
                "deployment_name": deployment_name,
                "pod_name": pod_name,

                "deployment_creation_worker2_date": deployment_creation_worker2_date,
                "deployment_schedule_worker2_date": deployment_schedule_worker2_date,
                "deployment_deletion_worker2_date": deployment_deletion_worker2_date,

                "selectivedeployment_creation_worker1_date": selectivedeployment_creation_worker1_date,
                "selectivedeployment_deletion_worker1_date": selectivedeployment_deletion_worker1_date,

                "selectivedeployment_creation_worker2_date": selectivedeployment_creation_worker2_date,
                "selectivedeployment_deletion_worker2_date": selectivedeployment_deletion_worker2_date,

                "pod_creation_date": pod_creation_date,
                "pod_schedule_date": pod_schedule_date,
                "pod_deletion_date": pod_deletion_date,
            }
            rows.append(row)
    
    return pd.DataFrame(rows)

In [125]:
df = edgenetfed_multi_deployment_to_dataframe("jsons/edgenetfed1/20230330_091022.json")